In [ ]:
# Use "jupyter_plotly_dash" for Apporto.
# Use "jupyter_dash" for local machine.
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

#TODO
#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_Shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "userPass"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.readAll({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H5('Student: Preston Burkhardt'))),
    html.Center(html.B(html.H5('Class: CS-340-T6640 Client/Server Development 22EW6'))),

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_mode="multi",
        column_selectable='multi',
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6',)
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#This callback handles the map. Marker is displayed for the animal/row that is selected.
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "selected_rows"),
     Input('datatable-id', "page_size"),
     Input('datatable-id', "page_current")])
def update_map(viewData, selectedRow, currPage, currPageSize):
    #handles index errors by displaying map with no markers
    try:
        dff = pd.DataFrame.from_dict(viewData)
        selectedRow = selectedRow[0] - (currPage * currPageSize) #needed for index when page changes
    except IndexError:
        return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=8,
               children=[dl.TileLayer(id="base-layer-id")])]

    # map marker changes based on which row is selected
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[selectedRow,13],dff.iloc[selectedRow,14]], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[selectedRow,13],dff.iloc[selectedRow,14]], children=[
                dl.Tooltip(dff.iloc[selectedRow,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selectedRow,9])
                ])
            ])
        ])
    ]



# Use "app" for Apporto.
# Use "app.run_server()" for local machine.
#app
app.run_server(host="localhost",port=8051)